This notebook will use IFCB images to train a convolutional neural network and have it classify images.

In [ ]:
import matplotlib.pyplot as plt
%pylab inline
%matplotlib inline


from keras.models import Sequential, Model, load_model
from keras.layers import Dense, Activation, Conv2D, MaxPooling2D, Flatten, Dropout, BatchNormalization, ZeroPadding2D, Input
from keras.layers import concatenate
from keras.preprocessing import image as keras_image
from keras.optimizers import Adam
from keras.backend import tf as ktf
from keras.constraints import maxnorm
from keras.layers import Add, Multiply, Concatenate, Average

import keras.backend as K
import numpy as np
import cv2 as cv2
from collections import Counter

import os
import skimage.transform as ski_transform
import skimage.io as ski_io
from skimage import img_as_float


from PIL import Image as PIL_Image

import warnings
warnings.filterwarnings('ignore')

from sklearn.metrics import classification_report, confusion_matrix

import pandas as pd
import seaborn as sns
from scipy.io import loadmat, savemat

import ROI_image_reader_stitched as ROI
import shutil
import pickle
import ROI_image_reader_stitched

In [ ]:
import keras
keras.__version__

In [ ]:
home_path = 'F:/IFCB/'
out_folder_of_images = home_path + 'Training_sets_padded/'  #path to the padded images from the training set
out_folder_of_images_unpadded = home_path + 'Training_sets_unpadded/' #path to the unpadded images from the training set
folder_of_images_validation = home_path + 'validation_sets/' #where to put the validation sets

number_of_categories = 112

In [ ]:
def eliminate_background(image):
    shape = image.shape
    mid = int(shape[0] / 2)
    bkgd_mean = image[mid,:].mean()
    bkgd_std = image[mid, :].std()
    image -= bkgd_mean
    image /= (bkgd_std+0.001)
    
    image *= -1
    return image

In [ ]:
#image size
image_size = 300  #an X by X size square

In [ ]:
path_to_models = '/path/to/models/'

In [ ]:
#load all three models if possible
#combine them into one single ensemble model
#make each individual model untrainable
#train a final layer for weighting the individual models

model1 = load_model(path_to_models + 'CNN_model_mdl1_padded.mdl')
#model1.load_weights(path_to_models + 'CNN_model_weights_mdl1.wts')
model1.trainable = False

model2 = load_model(path_to_models + 'CNN_model_mdl2_padded.mdl')
#model2.load_weights(path_to_models + 'CNN_model_weights_mdl2.wts')
model2.trainable = False

model3 = load_model(path_to_models + 'CNN_model_mdl3_padded.mdl')
#model3.load_weights(path_to_models + 'CNN_model_weights_mdl3.wts')
model3.trainable = False

model4 = load_model(path_to_models + 'CNN_model_mdl1_unpadded.mdl')
##model4.load_weights(path_to_models + 'CNN_model_weights_mdl3.wts')
model4.trainable = False

model5 = load_model(path_to_models + 'CNN_model_mdl2_unpadded.mdl')
#model2.load_weights(path_to_models + 'CNN_model_weights_mdl2.wts')
model5.trainable = False

model6 = load_model(path_to_models + 'CNN_model_mdl3_unpadded.mdl')
#model3.load_weights(path_to_models + 'CNN_model_weights_mdl3.wts')
model6.trainable = False

In [ ]:
for num, layer in enumerate(model1.layers):
    layer.name = 'Model1_'+str(num)
model1.name = 'Model1'

for num, layer in enumerate(model2.layers):
    layer.name = 'Model2_'+str(num)
model2.name =  'Model2'

for num, layer in enumerate(model3.layers):
    layer.name = 'Model3_'+str(num)
model3.name = 'Model3'

for num, layer in enumerate(model4.layers):
    layer.name = 'Model4_'+str(num)
model4.name = 'Model4'

for num, layer in enumerate(model5.layers):
    layer.name = 'Model5_'+str(num)
model5.name =  'Model5'

for num, layer in enumerate(model6.layers):
    layer.name = 'Model6_'+str(num)
model6.name = 'Model6'

In [ ]:

padded_inputs = Input((image_size, image_size, 1))
unpadded_inputs = Input((image_size, image_size, 1))

out1 = model1(padded_inputs)
out2 = model2(padded_inputs)
out3 = model3(padded_inputs)
#out4 = model4(common_inputs)
out4 = model4(unpadded_inputs)
out5 = model5(unpadded_inputs)
out6 = model6(unpadded_inputs)

test = Concatenate()([out1, out2, out3, out4, out5, out6])
test = Dense(112, name='model_ensemble_3')(test)
test = Activation('softmax', name='model_ensemble_4')(test)

ensemble = Model([padded_inputs, unpadded_inputs], test)


In [ ]:
adam = Adam(lr=0.00001, decay=.000001)
ensemble.compile(loss='categorical_crossentropy',
              optimizer=adam,
              metrics=['accuracy'],)

In [ ]:
ensemble.summary()

In [ ]:
def get_image_batches(num_batches=6795):
    padded = []
    unpadded = []
    answers = []
    for x in range(num_batches):
        if x % 50 == 0:
            print(x, end=',')
        temp1 = photos.next()
        temp2 = photos_unpadded.next()
        padded.extend(temp1[0])
        unpadded.extend(temp2[0])
        answers.extend(temp1[1])
    
    return [padded, unpadded, answers]

In [ ]:
def get_image_batch_generator():
    
    while 1:
        temp1 = photos.next()
        temp2 = photos_unpadded.next()
        
    
        yield [[temp1[0], temp2[0]], temp1[1]]
        
    return

def get_image_batch_generator_prediction():
    
    while 1:
        temp1 = photos.next()
        temp2 = photos_unpadded.next()
        
    
        yield [temp1[0], temp2[0]]
        
    return

In [ ]:
#load the ensemble model
ensemble.load_weights(path_to_models + 'ensemble_model__weights.wts')

In [ ]:
import time
# this version modified to classify images in a batch format (ideally 16 images at a time * 4 versions of each image)
#this version modified to output class files
#this version modified to use a different threshold for each category
def classify_images_merged_pad_nopad(images, filename, outpath, class2use, cnn_model, generate_images=False, classification_threshold=None, pad_images=True):
    image_results = {'TBscores':[],
                     'roinum':[],
                     'TBclass':[],
                     'TBclass_above_threshold':[],
                     'class2useTB':[],
                     '__header__':'Python 3 Keras CNN',
                     '__version__':'1.0',
                     '__globals__':[]
                    }
    time1 = time.time()
    if not classification_threshold:
        classification_threshold = [0.38 for category in class2use]
    elif len(classification_threshold) == 1:
        temp_threshold = classification_threshold[0]
        classification_threshold = [temp_threshold for category in class2use]
        
    num_images = len(images)
    batch_size = 16
    num_orientations = 2  # between 1-4 how many flips to the image for classifying it multiple ways 
    batches = int(num_images / batch_size) #how many batches to run
    leftover_images = int(num_images % batch_size) #what the last batch size will be
    img_num = 0
    all_img_predictions = []
    #create the batches
    for batch in range(batches): #was for image in images
        imgs_to_classify = []
        imgs_to_classify_nopad = []
        
        for batch_image in range(batch_size):
            #padded images
            input_image = resize_image3(np.array(images[img_num][1]), pad_images=True)
            input_image = eliminate_background(input_image)
            input_image *= (1/255.) #classifier was trained using data that had been scaled to 0-1 from 0-255
            imgs_to_classify.append(input_image)
            if num_orientations > 1: 
                imgs_to_classify.append(input_image[::-1])
                
                if num_orientations > 2:
                    imgs_to_classify.append(input_image[:,::-1])
                    
                    if num_orientations > 3:
                        imgs_to_classify.append(input_image[::-1,::-1])
            
            #unpadded images
            input_image = resize_image3(np.array(images[img_num][1]), pad_images=False)
            input_image = eliminate_background(input_image)
            input_image *= (1/255.) #classifier was trained using data that had been scaled to 0-1 from 0-255
            imgs_to_classify_nopad.append(input_image)
            if num_orientations > 1: 
                imgs_to_classify_nopad.append(input_image[::-1])
                
                if num_orientations > 2:
                    imgs_to_classify_nopad.append(input_image[:,::-1])
                    
                    if num_orientations > 3:
                        imgs_to_classify_nopad.append(input_image[::-1,::-1])
            
            img_num += 1
        imgs_to_classify = np.array(imgs_to_classify).reshape(batch_size*num_orientations, 300, 300, 1)
        imgs_to_classify_nopad = np.array(imgs_to_classify_nopad).reshape(batch_size*num_orientations, 300, 300, 1)

        ##trying to predict in a batch instead of singly; this has [batch_size] images each of which is in two variations
        probs = cnn_model.predict([imgs_to_classify, imgs_to_classify_nopad], batch_size=batch_size*num_orientations)
        for prob in range(0,probs.shape[0],num_orientations):
            all_img_predictions.append(probs[prob:prob+num_orientations].mean(axis=0).astype('float16'))
        
    #now do the remaining images if necessary
    if leftover_images > 0:
        imgs_to_classify = []
        imgs_to_classify_nopad = []
        
        for batch_image in range(leftover_images):
            #padded images
            input_image = resize_image3(np.array(images[img_num][1]), pad_images=True)
            input_image = eliminate_background(input_image)
            input_image *= (1/255.) #classifier was trained using data that had been scaled to 0-1 from 0-255
            imgs_to_classify.append(input_image)
            if num_orientations > 1: 
                imgs_to_classify.append(input_image[::-1])
                
                if num_orientations > 2:
                    imgs_to_classify.append(input_image[:,::-1])
                    
                    if num_orientations > 3:
                        imgs_to_classify.append(input_image[::-1,::-1])
            
            #unpadded images
            input_image = resize_image3(np.array(images[img_num][1]), pad_images=False)
            input_image = eliminate_background(input_image)
            input_image *= (1/255.) #classifier was trained using data that had been scaled to 0-1 from 0-255
            imgs_to_classify_nopad.append(input_image)
            if num_orientations > 1: 
                imgs_to_classify_nopad.append(input_image[::-1])
                
                if num_orientations > 2:
                    imgs_to_classify_nopad.append(input_image[:,::-1])
                    
                    if num_orientations > 3:
                        imgs_to_classify_nopad.append(input_image[::-1,::-1])
            
            
            img_num += 1

        imgs_to_classify = np.array(imgs_to_classify).reshape(leftover_images*num_orientations, 300, 300, 1)
        imgs_to_classify_nopad = np.array(imgs_to_classify_nopad).reshape(leftover_images*num_orientations, 300, 300, 1)

            ##trying to predict in a batch instead of singly; this has [batch_size] images each of which is in four variations
        probs = cnn_model.predict([imgs_to_classify, imgs_to_classify_nopad], batch_size=leftover_images)
        for prob in range(0,probs.shape[0],num_orientations):
            all_img_predictions.append(probs[prob:prob+num_orientations].mean(axis=0).astype('float16'))
    
    #classification through network is complete
    time2 = time.time()
    img_num = 0
    for img_prob in all_img_predictions:
        predicted = class2use[np.argmax(img_prob)] #where does check_answer come in; replace with class2use
        #add the general information to the class file
        image_results['roinum'].append([images[img_num][0]])
        image_results['TBscores'].append(img_prob)
        image_results['TBclass'].append([np.array(predicted)])
        if img_prob.max() >= classification_threshold[np.argmax(img_prob)]:
            image_results['TBclass_above_threshold'].append([np.array(predicted)])
            if generate_images:
                ski_io.imsave('{0}{1}/{2}_{3:05d}.png'.format(out_folder_of_clustered_images, predicted, filename, images[img_num][0]), np.array(images[img_num][1]))
                
        else:
            image_results['TBclass_above_threshold'].append([np.array('unclassified')])
            if generate_images:
                ski_io.imsave('{0}{1}/{2}_{3:05d}.png'.format(out_folder_of_clustered_images, 'unclassified' ,filename, images[img_num][0]), np.array(images[img_num][1]))
        img_num += 1
        
    time3 = time.time()    
    ##code below should write the class file
    image_results['TBscores'] = np.array(image_results['TBscores'])
    image_results['TBclass_above_threshold'] = np.array(image_results['TBclass_above_threshold'])
    
    image_results['roinum'] = np.array(image_results['roinum']).astype('uint16')
    temp_class = np.empty((len(image_results['TBclass']),), object)
    temp_class_threshold = np.empty((len(image_results['TBclass_above_threshold']),), object)
    for img in range(len(temp_class)):
        temp_class[img] = image_results['TBclass'][img][0]
        temp_class_threshold[img] = image_results['TBclass_above_threshold'][img][0]
    image_results['TBclass'] = temp_class
    image_results['TBclass_above_threshold'] = temp_class_threshold
    #create the class2use variable
    temp_class2use = np.empty((len(class2use),), object)
    for ind_class in range(len(class2use)):
        temp_class2use[ind_class] = np.array(class2use[ind_class])
    image_results['class2useTB'] = temp_class2use
    savemat(outpath+filename+'_class_vCNN1.mat', image_results, do_compression=True)
    
    time4 = time.time()
    print()
    print('Classification:  ', time2-time1)
    print('Thresholding:    ', time3-time2)
    print('Write class file:', time4-time3)
    return 

In [ ]:
#load the probability results in case we want to change our threshold later
with open('/path/to/models/thresholds_ensemble.pck', 'rb') as f:
    thresholds = pickle.load(f)
    

In [ ]:
[(check_answer[x], thresholds[x]) for x in range(112)]

In [ ]:
#creating new function to mimic how the training set images were resized and centered
def resize_image3(temp_image, pad_images=True):
    image_size = 300
    temp_shape = temp_image.shape
    bkgd_mean = temp_image[0,:].mean()
    if pad_images:
        if temp_image.shape[0] != temp_image.shape[1]:
            if temp_image.shape[0] > image_size or temp_image.shape[1] > image_size:
                new_image = np.full((max(temp_image.shape), max(temp_image.shape)), bkgd_mean)
            else:
                new_image = np.full((image_size, image_size), bkgd_mean)
            new_shape = new_image.shape
            center = np.array(new_shape) - np.array(temp_image.shape)
            new_image[int(center[0]/2):(int(center[0]/2) + temp_shape[0]), int(center[1]/2):(int(center[1]/2) + temp_shape[1])] = temp_image           
        else:
            new_image = ski_transform.resize(temp_image, (image_size, image_size))
        if new_image.shape[0] > image_size:
            new_image = ski_transform.resize(new_image, (image_size, image_size))
    else:
        new_image = ski_transform.resize(temp_image, (image_size, image_size))
    #print('new_image = '+str(new_image))
    return img_as_float(new_image)# * (1/255.))

def load_IFCB_file(filename, datapath):
    images = ROI_image_reader_stitched.process_file(datapath+filename)
    return images


In [ ]:
#put the folders in the out directory
#add in an unclassified directory
class2use_master_out = loadmat('/path/to/class2use_master/file/') #load the class2use_master file to get the class names
class2use = [class2use_master_out['class2use'][0][x][0] for x in range(len(class2use_master_out['class2use'][0]))]

#where should the image folders go?
out_folder_of_clustered_images = '/path/to/out/images/'
check_answer = sort(list(class2use))
for x in check_answer:
    os.mkdir(out_folder_of_clustered_images + str(x))
if 'unclassified' not in check_answer:
    os.mkdir(out_folder_of_clustered_images + 'unclassified')

In [ ]:
datapath = '/path/to/raw/IFCB/data/' #where are the ROI, HDR, ADC files?
listofdirs = sort(os.listdir(datapath))
outpath = '/path/to/class/files/' #where should the class_file outputs be placed?
finished_files = os.listdir(outpath)
for indir in listofdirs[::-1]:
    if indir == 'beads': #skip the beads folder if present
        continue
    listoffiles_temp = os.listdir(datapath + indir)
    listoffiles = sort(list(set([x[:-4] for x in listoffiles_temp])))
    for filenum, datafile in enumerate(listoffiles[:]):
        if datafile + '_class_CNNv1.mat' in finished_files:
            continue
        else:
            print('Starting {0} {1}...loading file'.format(filenum, datafile), end='...')
            try:
                test_data = load_IFCB_file(datafile, datapath+indir+'/')
                print('Num images:{0}'.format(len(test_data)))
                print('classifying images', end='...')
                classify_images_merged_pad_nopad(test_data, datafile, outpath, class2use, ensemble, generate_images=False,
                                      classification_threshold=thresholds, pad_images=True)
                print('done!')
            except:
                print('error occurred!')